In [32]:
 # Dependencies
import requests
import pandas as pd
import json
from pprint import pprint
from pandas.io.json import json_normalize

In [33]:
# Create dataframe of JSON data
scicenter_DF = pd.read_csv('Webscraper/science_center_data.csv',encoding='Latin-1')
scicenter_DF.head()

,Unnamed: 0,address_line_one,address_line_three,address_line_two,full_address,name,state,website,latitude,longitude
0,0,625 C St,United States,"Anchorage, AK","625 C St Anchorage, AK United States",Anchorage Museum at Rasmuson Center,AK,http://www.anchoragemuseum.org,61.215669,-149.884906
1,1,PO Box 1587,United States,"Anniston, AL","PO Box 1587 Anniston, AL United States",Anniston Museums and Gardens,AL,http://www.exploreamag.org,33.659826,-85.831632
2,2,PO Box 1968,United States,"Mobile, AL","PO Box 1968 Mobile, AL United States",Gulf Coast Exploreum Science Center,AL,http://www.exploreum.com,30.695366,-88.039891
3,3,PO Box 3005,United States,"Mobile, AL","PO Box 3005 Mobile, AL United States",GulfQuest (National Maritime Museum of the Gul...,AL,http://www.nationalmaritime.us,30.695366,-88.039891
4,4,PO Box 1507,United States,"Gadsden, AL","PO Box 1507 Gadsden, AL United States",Mary G. Hardin Center for Cultural Arts,AL,http://www.culturalarts.com,34.014264,-86.006639


In [34]:
scicenter_DF = scicenter_DF.rename(columns={'address_line_one': 'Address',
                                                            'name': 'Name',
                                                            'state': 'State',
                                                            'website': 'Website'})
scicenter_DF.head()

,Unnamed: 0,Address,address_line_three,address_line_two,full_address,Name,State,Website,latitude,longitude
0,0,625 C St,United States,"Anchorage, AK","625 C St Anchorage, AK United States",Anchorage Museum at Rasmuson Center,AK,http://www.anchoragemuseum.org,61.215669,-149.884906
1,1,PO Box 1587,United States,"Anniston, AL","PO Box 1587 Anniston, AL United States",Anniston Museums and Gardens,AL,http://www.exploreamag.org,33.659826,-85.831632
2,2,PO Box 1968,United States,"Mobile, AL","PO Box 1968 Mobile, AL United States",Gulf Coast Exploreum Science Center,AL,http://www.exploreum.com,30.695366,-88.039891
3,3,PO Box 3005,United States,"Mobile, AL","PO Box 3005 Mobile, AL United States",GulfQuest (National Maritime Museum of the Gul...,AL,http://www.nationalmaritime.us,30.695366,-88.039891
4,4,PO Box 1507,United States,"Gadsden, AL","PO Box 1507 Gadsden, AL United States",Mary G. Hardin Center for Cultural Arts,AL,http://www.culturalarts.com,34.014264,-86.006639


In [35]:
scicenter_DF = scicenter_DF[~scicenter_DF['State'].str.contains('PR')]
scicenter_DF = scicenter_DF[~scicenter_DF['State'].str.contains('HI')]
scicenter_DF = scicenter_DF[~scicenter_DF['State'].str.contains('AK')]
scicenter_DF.head()

,Unnamed: 0,Address,address_line_three,address_line_two,full_address,Name,State,Website,latitude,longitude
1,1,PO Box 1587,United States,"Anniston, AL","PO Box 1587 Anniston, AL United States",Anniston Museums and Gardens,AL,http://www.exploreamag.org,33.659826,-85.831632
2,2,PO Box 1968,United States,"Mobile, AL","PO Box 1968 Mobile, AL United States",Gulf Coast Exploreum Science Center,AL,http://www.exploreum.com,30.695366,-88.039891
3,3,PO Box 3005,United States,"Mobile, AL","PO Box 3005 Mobile, AL United States",GulfQuest (National Maritime Museum of the Gul...,AL,http://www.nationalmaritime.us,30.695366,-88.039891
4,4,PO Box 1507,United States,"Gadsden, AL","PO Box 1507 Gadsden, AL United States",Mary G. Hardin Center for Cultural Arts,AL,http://www.culturalarts.com,34.014264,-86.006639
5,5,200 19th St N,United States,"Birmingham, AL","200 19th St N Birmingham, AL United States",McWane Science Center,AL,http://www.mcwane.org,33.514892,-86.808403


In [36]:
CityState = scicenter_DF['address_line_two'].str.split(',', expand = True)

# Create columns for these split values of County and State
scicenter_DF['City'] = CityState[0]

In [37]:
# Write function to create geoJSON dictionary from dataframe
def df_to_geojson(df, properties):
    
    # Create Python dictionary to hold geoJSON information
    geojson = {'type':'FeatureCollection', 'features':[]}
    
    # Loop through the rows of the dataframe
    for _, row in df.iterrows():
        
        # Create feature for each object
        feature = {'type':'Feature',
                   'properties':{},
                   'geometry':{'type':'Point',
                               'coordinates':[[],[]]}}
        
        # Place geometry coordinates into geometry
        feature['geometry']['coordinates'][0] = row['latitude']
        feature['geometry']['coordinates'][1] = row['longitude']
        
        # Go through each item in columns list and append that to the properties section
        for prop in properties:
            feature['properties'][prop] = row[prop]
            
        # Append feature to dictionary
        geojson['features'].append(feature)
        
    # Return geoJSON dictionary
    return geojson

In [38]:
# Define list of properties
col = ["Address", "City", "State", "Name", "Website" ]
#col = ["Address", "City", "State", "ame", "website" ]
# Use function to Create geoJSON file
new_geoJSON = df_to_geojson(scicenter_DF, col)

# Print geoJSON file
new_geoJSON

{'type': 'FeatureCollection',
 'features': [{'type': 'Feature',
   'properties': {'Address': 'PO Box 1587',
    'City': 'Anniston',
    'State': 'AL',
    'Name': 'Anniston Museums and Gardens',
    'Website': 'http://www.exploreamag.org'},
   'geometry': {'type': 'Point',
    'coordinates': [33.659825700000006, -85.8316318]}},
  {'type': 'Feature',
   'properties': {'Address': 'PO Box 1968',
    'City': 'Mobile',
    'State': 'AL',
    'Name': 'Gulf Coast Exploreum Science Center',
    'Website': 'http://www.exploreum.com'},
   'geometry': {'type': 'Point', 'coordinates': [30.6953657, -88.0398912]}},
  {'type': 'Feature',
   'properties': {'Address': 'PO Box 3005',
    'City': 'Mobile',
    'State': 'AL',
    'Name': 'GulfQuest (National Maritime Museum of the Gulf of Mexico)',
    'Website': 'http://www.nationalmaritime.us'},
   'geometry': {'type': 'Point', 'coordinates': [30.6953657, -88.0398912]}},
  {'type': 'Feature',
   'properties': {'Address': 'PO Box 1507',
    'City': 'Gads

In [39]:
# Create new file named data.json
with open('scicenter.json', 'w') as fp:
    
    # format as JSON file from dictionary and indent as needed
    json.dump(new_geoJSON, fp)